## 2x2 Vectors Graph

A simple model with two input layers and two output layers.

## Build and save the graph

Ensure you are working in a virtual environment with tensorflow >= 1.9 installed, and in the notebook's directory on the command line:

```bash
rm -r saved_graph
```

In [2]:
import tensorflow as tf
import os

In [3]:
# construct the graph

X = tf.placeholder(tf.float32, shape=(4,4), name="input1")
Y = tf.placeholder(tf.float32, shape=(4,4), name="input2")

W_in = [[1,0,1,0], [0,1,0,1], [1,0,1,0], [0,1,0,1]]
B_in = [[0,1,0,1], [1,0,1,0], [0,1,0,1], [1,0,1,0]]

W = tf.Variable(W_in, dtype=tf.float32, name="W")
B = tf.Variable(B_in, dtype=tf.float32, name="B")

XW = tf.matmul(X, tf.transpose(W))
YB = tf.matmul(Y, tf.transpose(B))

Z = tf.multiply( XW, YB, name="output1" )
S = tf.add( XW, YB, name="output2" )

# execute the graph

init = tf.global_variables_initializer()
saver = tf.train.Saver()

X_in = [[1,2,3,4], [10,20,30,40], [100,200,300,400], [1000,2000,3000,4000]]
Y_in = [[5,6,7,8], [50,60,70,80], [500,600,700,800], [5000,6000,7000,8000]]

with tf.Session() as sess:
    sess.run(init)
    Z_eval, S_eval = sess.run([Z, S], feed_dict={X: X_in, Y: Y_in})
    
    # saver.save(sess, "./saved_checkpoints/model.ckpt")
    # tf.train.write_graph(sess.graph, './saved_graph', 'graph.pb', as_text=False)
    tf.saved_model.simple_save(sess, "./saved_graph", inputs={"input1": X, "input2": Y}, outputs={"output1": Z, "output2": S})
    
    print(Z_eval)
    print(S_eval)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_graph/saved_model.pb
[[5.6e+01 7.2e+01 5.6e+01 7.2e+01]
 [5.6e+03 7.2e+03 5.6e+03 7.2e+03]
 [5.6e+05 7.2e+05 5.6e+05 7.2e+05]
 [5.6e+07 7.2e+07 5.6e+07 7.2e+07]]
[[   18.    18.    18.    18.]
 [  180.   180.   180.   180.]
 [ 1800.  1800.  1800.  1800.]
 [18000. 18000. 18000. 18000.]]


## Freeze The Graph

Having used the `tf.saved_model.simple_save` function, now use the `freeze_graph` cli. Tensorflow also provides a python interface to the freeze_graph function.

On the command line:

```bash
mkdir frozen_graph

freeze_graph \
    --input_saved_model_dir=saved_graph \
    --saved_model_tags=serve \
    --output_graph=frozen_graph/frozen_graph.pb \
    --output_node_names=output1,output2
```

## Convert the Model to a TensorFlow Lite Format

Use the `toco` command line utility to convert the frozen graph to a tensorflow lite compatible file:

```bash
mkdir tflite

toco \
	--graph_def_file=frozen_graph/frozen_graph.pb \
	--output_file=tflite/model.tflite \
	--input_arrays=input1,input2 \
	--input_shapes=4,4:4,4 \
	--output_arrays=output1,output2
```